<a href="https://colab.research.google.com/github/juboyy/Groq-Self-Learning/blob/main/groq-selfLearn-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# pip é como uma loja de aplicativos, mas em vez de baixar apps, ele baixa ferramentas e componentes que os programadores usam para construir programas.
# !pip install é o comando que usamos para dizer ao Python: "Baixe e instale essas ferramentas para mim, por favor!"
# Neste caso específico, estamos pedindo para instalar várias ferramentas relacionadas à criação de programas de Inteligência Artificial (IA) e automação, aqui é onde você instala seus modelos.

!pip install langchain_openai langchain-google-genai langchain-groq langchain_core python-dotenv langchain langchain_community

ERROR: Operation cancelled by user




---


### **1. Importações de Ferramentas**
**ChatGroq**: Estes são como "conversadores de IA" que podem responder a perguntas ou ajudar a realizar tarefas com inteligência artificial. Eles permitem conectar e conversar com sistemas de IA diferentes, como o ChatGPT (OpenAI) e o modelo Groq.

  **`ChatMessageHistory`**: Mantém um histórico das conversas, como se fosse um diário de tudo o que foi discutido até agora com o "conversador de IA".

  **`ChatPromptTemplate`** e **`MessagesPlaceholder`**: Estas são ferramentas para formatar mensagens e diálogos que você terá com a IA, ajudando a configurar como a conversa deve acontecer.


---


### **2. Carregar Variáveis de Configuração**
`from dotenv import load_dotenv`: Esta ferramenta permite carregar informações confidenciais (como senhas ou chaves de acesso) de um arquivo seguro, para que o código possa usá-las sem expor esses dados diretamente.
`os.environ["GROQ_API_KEY"]`: Configura a chave de acesso para o sistema de IA Groq. Isso é como dizer à IA: "Aqui está minha chave; agora podemos conversar."


---



In [5]:
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

## **Vamos definir funções de apoio para cada habilidade do nosso agente**

Esta função obtém o histórico de conversas para uma sessão específica.
    
Parâmetros:
- store: Um dicionário que guarda os históricos de conversa para diferentes sessões.
- session_id: Uma string que identifica de forma única uma sessão de usuário.
    
Retorna:
- O histórico de mensagens associado ao session_id fornecido.
  Se não existir um histórico para essa sessão, ele cria um novo.

In [6]:
# Gerenciamento de Histórico do chat

def get_chat_history(store, session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


Esta função gera uma resposta com base na entrada do usuário e em informações adicionais.

Parâmetros:
- chain_with_history: Um objeto que mantém o histórico da conversa.
- human_input (str): A mensagem que o usuário enviou.
- session_id (str): Um identificador único para acompanhar a sessão do usuário.
- insights (str): Informações que ajudam a melhorar a resposta.

Retorna:
- A resposta gerada como uma string.


In [7]:
# Gerar Resposta

def generate_response(chain_with_history, human_input: str, session_id: str, insights: str):
    response = chain_with_history.invoke(
        {"input": human_input, "insights": insights},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

Esta função analisa o histórico de conversas e fornece insights para melhorar as respostas.

Parâmetros:
- llm: Modelo de linguagem utilizado para gerar a reflexão.
- store: Dicionário que armazena o histórico das conversas.
- session_id: Identificador único da sessão.

Retorna:
- Conteúdo da reflexão gerada.

In [8]:
# Reflexão
def reflect(llm, store, session_id: str):
    reflection_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on the following conversation history, provide insights on how to improve responses:"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "Generate insights for improvement:")
    ])
    reflection_chain = reflection_prompt | llm
    history = get_chat_history(store, session_id)
    reflection_response = reflection_chain.invoke({"history": history.messages})
    return reflection_response.content

Esta função ajuda o agente a aprender com base em insights fornecidos.
Ela cria um prompt que instrui o agente a atualizar seu conhecimento e comportamento com base nesses insights,
resume os 10 pontos-chave relevantes e realiza uma pesquisa básica sobre cada tópico.

Em seguida, o agente adiciona uma mensagem ao histórico da conversa indicando o que foi aprendido.

In [24]:
# Aprendizado
def learn(llm, store, session_id: str, insights: str):
    learning_prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on these insights, update the agent's knowledge and behavior:"),
        ("human", "{insights}"),
        ("human", "Summarize the 10 relevant key points to remember and provide a basic research about each topic:")
    ])
    learning_chain = learning_prompt | llm
    learned_points = learning_chain.invoke({"insights": insights}).content
    get_chat_history(store, session_id).add_ai_message(f"[SYSTEM] Agent learned: {learned_points}")
    return learned_points

 A classe SelfImprovingAgent é uma implementação de um agente de IA auto-aprimorável em Python.
 Ela utiliza um modelo de linguagem (LLM) da Groq para processar e gerar respostas.

 A classe é inicializada com um LLM, um dicionário para armazenar o histórico de conversas e uma string para insights.

 O prompt do agente é configurado para aprender e melhorar ao longo do tempo, respondendo em português.

 A classe usa um sistema de cadeia (chain) para processar as entradas, que inclui o histórico da conversa.

 Os métodos principais são: 'respond' para gerar respostas, 'reflect' para analisar o histórico e gerar insights, e 'learn' para aplicar esses insights e melhorar o desempenho do agente.

 Esta estrutura permite que o agente melhore continuamente suas respostas com base em interações anteriores.


In [27]:
# Auto-aprendizado Agent Class
# vamos definir nossa classe de agente que utiliza todas estas funções acima

class SelfImprovingAgent:
    def __init__(self):
        self.llm = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", max_tokens=2000, temperature=0.7)
        self.store = {}
        self.insights = ""

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a self-improving AI assistant. Learn from your interactions and improve your performance over time. apenas responda em pt-br"),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}"),
            ("system", "Recent insights for response improvement: {insights}")
        ])

        self.chain = self.prompt | self.llm
        self.chain_with_history = RunnableWithMessageHistory(
            self.chain,
            lambda session_id: get_chat_history(self.store, session_id),
            input_messages_key="input",
            history_messages_key="history"
        )

    def respond(self, human_input: str, session_id: str):
        return generate_response(self.chain_with_history, human_input, session_id, self.insights)

    def reflect(self, session_id: str):
        self.insights = reflect(self.llm, self.store, session_id)
        return self.insights

    def learn(self, session_id: str):
        self.reflect(session_id)
        return learn(self.llm, self.store, session_id, self.insights)


### **Hora de testar!**

In [29]:
agent = SelfImprovingAgent()
session_id = "user_123"


# 1 interação
print("AI:", agent.respond("What's the capital of Brazil?", session_id))

# 2 interação
print("AI:", agent.respond("Can you tell me more about its history?", session_id))

# interação Learn and improve
print("\nReflecting and learning...")
learned = agent.learn(session_id)
print("Learned:", learned)

#3 interação (potentially improved based on learning)
print("\nAI:", agent.respond("qual a capital do brasil?", session_id))

#4 interação (to demonstrate continued improvement)
print("AI:", agent.respond("What's another interesting fact about this city?", session_id))

#5 interação (to demonstrate continued improvement)
print("AI:", agent.respond("me diga as maiores curiosidades?", session_id))


AI: A capital do Brasil é Brasília.
AI: Brasília é a capital do Brasil desde 1960. Foi projetada e construída para ser a capital do país, substituindo a então capital, Rio de Janeiro. A cidade foi inaugurada em 21 de abril de 1960, e desde então tem se tornado um importante centro político, cultural e econômico do país.

Reflecting and learning...
Learned: 1. Improve response clarity: Ensure that responses are detailed and clear, providing enough context for users to understand the information.

2. Use simpler language: Utilize simpler language or provide translations to make information accessible to a wider audience.

3. Add more relevant information: Include comprehensive details about the topic, such as history, cultural significance, or its role in a particular context.

4. Consider user's perspective: Tailor responses based on the user's interests and perspective to make them more relevant and engaging.

5. Provide context: Ensure that responses include necessary context to help 